In [1]:
import pandas as pd
import numpy as np

from utils.utils import split_dataset

from sktime.utils.load_data import from_long_to_nested
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
full_array = np.load('data/Foch_transplant.npy', allow_pickle=True)
col = np.load('data/Foch_transplant_name_col.npy', allow_pickle=True)

In [3]:
df = pd.DataFrame(full_array, columns=col)

In [4]:
%%time
dic1 = {'case_id':[], 'dim_id':[], 'reading_id':[], 'value':[]}
nb_max = df['SpO2_by_FiO2'].apply(len).max()

# keep only time series
for k, col in enumerate(['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']):
    
    for i, arr in enumerate(df[col]):
        l = len(arr)
        for j, val in enumerate(arr):

            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [j]
            dic1['value']+= [val]
        # get ts same length
        for h in range(nb_max-l):
            dic1['case_id']+= [i]
            dic1['dim_id']+= [k]
            dic1['reading_id']+= [l+h]
            dic1['value']+= [0]

Wall time: 6.32 s


In [5]:
%%time
ddff1 = pd.DataFrame(dic1)
X_nested1 = from_long_to_nested(ddff1)

X_nested1.columns = ['B.I.S', 'BIS SR', 'DC', 'ETCO2', 'FC', 'FR',
       'FiO2', 'PAPdia', 'PAPmoy', 'PAPsys', 'PASd', 'PASm', 'PASs',
       'PEEPtotal', 'PNId', 'PNIm', 'PNIs', 'Pmax', 'Pmean', 'SpO2',
       'SvO2 (m)', 'Temp', 'VT', 'SpO2_by_FiO2', 'PASs_minus_PASd',
       'PAPsys_minus_PAPdia', 'declampage_cote1_done',
       'declampage_cote2_done']

Wall time: 23min 36s


# Export Array

In [6]:
def to_tensor(ss):
    shape = ss.shape[0]
    a = ss[0].to_numpy().reshape(-1,1)
    for s in ss[1:]:
        a = np.concatenate([a, s.to_numpy().reshape(-1,1)], axis=1)
    return a.reshape(1,1082, shape)

def get_tensor_format(X):
    dataframe = X.apply(to_tensor, axis=1)
    arr = dataframe[0]
    for a in dataframe[1:]:
        arr = np.concatenate([arr,a])
    return arr

## Full

In [7]:
%%time
X = get_tensor_format(X_nested1)
Y = df['target'].ravel()

Wall time: 6.62 s


In [8]:
X.shape

(330, 1082, 28)

In [9]:
col = np.array(X_nested1.columns)

x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true = split_dataset(X,Y,val_prop=0.1)

to_return = [x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true, col]

In [10]:
np.save('data/transplant_full.npy', to_return, allow_pickle=True)

## 10col

In [11]:
%%time
X = get_tensor_format(X_nested1[['BIS SR', 'ETCO2', 'FC', 'FR',
'FiO2','PAPsys', 'PASm',
'PEEPtotal', 'SpO2','SpO2_by_FiO2']])

Y = df['target'].ravel()

Wall time: 2.4 s


In [12]:
X.shape

(330, 1082, 10)

In [13]:
col = ['BIS SR', 'ETCO2', 'FC', 'FR',
'FiO2','PAPsys', 'PASm',
'PEEPtotal', 'SpO2','SpO2_by_FiO2']

x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true = split_dataset(X,Y, val_prop=0.1)

to_return = [x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true, col]

In [14]:
np.save('data/transplant_10col.npy', to_return, allow_pickle=True)

## 1col_concat

In [15]:
%%time
X = get_tensor_format(X_nested1[['BIS SR', 'ETCO2', 'FC', 'FR',
'FiO2','PAPsys', 'PASm',
'PEEPtotal', 'SpO2','SpO2_by_FiO2']])

X = X.reshape(330,10820)
Y = df['target'].ravel()

Wall time: 2.45 s


In [16]:
X.shape

(330, 10820)

In [17]:
col = ['time_series']

x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true = split_dataset(X,Y, val_prop=0.1)

to_return = [x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true, col]

In [18]:
np.save('data/transplant_1col_concat.npy', to_return, allow_pickle=True)

## 1col

In [30]:
%%time
X = get_tensor_format(X_nested1[['SpO2_by_FiO2']]).reshape(330,1082)
Y = df['target'].ravel()

Wall time: 232 ms


In [31]:
X.shape

(330, 1082)

In [32]:
col = ['SpO2_by_FiO2']

x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true = split_dataset(X,Y, val_prop=0.1)

to_return = [x_train,y_train,x_val,y_val,y_val_true,x_test,y_test,y_test_true, col]

In [33]:
np.save('data/transplant_1col.npy', to_return, allow_pickle=True)